In [15]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
import PIL
import os
import openCV as 

In [59]:
data_dir = 'E:\ELL_Project\problem3\\'
lis_folders = os.listdir(data_dir+'.')[:-1]
# print(lis_folders)
X = []

for folder in lis_folders:
    lis_files = os.listdir(data_dir+folder+'\.')
    im = []
    for files in lis_files:
        img = '\\'.join([data_dir,folder,files])
        image = plt.imread(img)
        im.append(image.flatten().reshape(-1,1))
    im_np = np.array(im)
    X.append(im_np)
for i in X:
    print(i.shape)
    print(i[0].shape)



# X_train


(10000, 4096, 1)
(4096, 1)
(8954, 4096, 1)
(4096, 1)
(10000, 4096, 1)
(4096, 1)
(10000, 4096, 1)
(4096, 1)
(10000, 4096, 1)
(4096, 1)
(10000, 4096, 1)
(4096, 1)


In [82]:
X_edit = X[1]
X_edit = np.array(X_edit)
X_edit = X_edit[:,:,0]
X_mean = np.sum(X_edit, axis=0) / X_edit.shape[0]
X_mean = X_mean.reshape(1,-1)
print(X_mean.shape)
diff = X[0].shape[0]-X[1].shape[0]
oneer = np.ones((diff,1))
X_app = np.dot(oneer,X_mean)
X_updated = np.append(X_edit,X_app,axis=0)
print(X_updated.shape)

(1, 4096)
(10000, 4096)


In [88]:
X_dash = X
X_dash[1] = X_updated.reshape(-1,4096,1)
X_np = np.array(X_dash)[:,:,:,0]
print(X_np.shape)

(6, 10000, 4096)


In [ ]:
def compress_data(X):
    num_feats = X.shape[0]
    for i in range(num_feats):
        X_feat = X[i,:,:]
        

In [ ]:
def import_dataset(split_percent = 70):

    split_point = split_percent//10

    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]

    return(X_train,X_test,y_train,y_test)

In [ ]:
def feature_scaling(X_train):
    training_size = X_train.shape[0]
    X_mean = np.sum(X_train,axis=0) / training_size
    X_var = np.sqrt(np.sum((np.square(X_train-X_mean)),axis=0)/training_size)
    X_train_reg = (X_train - X_mean) / X_var

    return (X_mean,X_var,X_train_reg)


In [ ]:
def KNN(X_train,y_train,X_test,y_test,k_num,feat_scal=True):

    test_size = X_test.shape[0]
    train_size = X_train.shape[0]
    # X_m,X_v,X_r,y_m,y_v,y_r = feature_scaling(X_train,y_train)
    if feat_scal:
        X_m,X_v,X_r = feature_scaling(X_train)
        X_test_reg = (X_test-X_m)/X_v
    else:
        X_r = X_train
        X_test_reg = X_test
    # y_test_reg = (y_test-y_m)/y_v
    y_pred = np.array([])
    # print(y_pred)
    for i in range(test_size):
        sample_X = X_test_reg[i:i+1,:]
        sample_y = y_test[i,:]
        dist_vec = np.sqrt(np.sum(np.square(X_r-sample_X),axis=1))
        dist_vec = dist_vec.reshape((train_size,1))

        d = np.column_stack((dist_vec[:,:],y_train[:,0]))
        # print(d)

        sorted_dist = d[np.argsort(d[:, 0])]
        temp_distance_vector = sorted_dist[:k_num,0]
        distance_vector = temp_distance_vector.reshape(-1,1)
        # distance_vector = np.append(np.ones(distance_vector.shape),distance_vector)
        # print(distance_vector.shape)
        for j in np.arange(0,10,0.5):
            distance_vector = np.append(distance_vector,(temp_distance_vector**j).reshape(-1,1),axis=1)
        distance_vector = distance_vector[:,1:]
        # print(distance_vector.shape)
        ddd = np.sum(distance_vector,axis=0).reshape(1,-1)
        # print(ddd[0,0])
        # print(ddd.shape)

        y_val_vector = sorted_dist[:k_num,1].reshape(-1,1)
        # print(y_val_vector.shape)
        # print(distance_vector.shape)
        # print(ddd.shape)
        num1 = np.dot(np.transpose(y_val_vector),distance_vector) / ddd
        # print(num1.shape)
        # num1 = np.max(num1)
        if i==0:
            # print('check')
            y_pred = num1.reshape(1,-1)
        else:
            # print(y_pred.shape)
            y_pred = np.append(y_pred,num1.reshape(1,-1),axis=0)
            

    # print(y_pred.shape)
    return y_pred
    
        



In [ ]:
def accuracy(y_pred,y_test):

    # y_pred = y_pred.reshape(-1,1)
    print('MSE LOSS')
    mse_loss = np.sum(np.square((y_test-y_pred)),axis=0) / y_test.shape[0]
    print(mse_loss)
    print('............................................................................')
    print('MAE LOSS')
    mae_loss = np.sum(np.abs((y_test-y_pred)),axis=0) / y_test.shape[0]
    print(mae_loss)
    # print('............................................................................')

In [ ]:
X_train,X_test,y_train,y_test = import_dataset()
y_pred = KNN(X_train,y_train,X_test,y_test,50,False)
accuracy(y_pred,y_test)